In [1]:
# Imports
from urllib import * #urlib2 has been moved to several modules
from urllib.request import urlopen
import tweepy
import json
import requests
from pprint import pprint # used to make the output look more pretty

In [2]:
credentials = {}
credentials['api_key'] = 'n2dTtVbZuJwji0auruBGVTBwT'
credentials['api_secret_key'] = 'U1Er7hNUFtOwmPIz6BCr6w1B3o9GlKhg6loffJVIK0cFAFdkYl'
credentials['access_token'] = '987093510116532231-tiHjc16Ws3vcMZpNUt0kuL7xj8iLSy1'
credentials['access_token_secret'] = 'ykZmkTz2TIeSajyWDKXvSSxAiFZvMFdD9ePUe0zn5fsna'

# Load Twitter API keys
api_key = credentials['api_key']
api_secret_key = credentials['api_secret_key']
access_token = credentials['access_token']
access_token_secret = credentials['access_token_secret']

# Twitter initialization
auth = tweepy.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [3]:
def insert(data, db):
    #GET method
    payload = data
    url='http://localhost:8983/solr/'+db+'/update/json/docs'
    headers = {'content-type': 'application/json'}
    r = requests.post(url, data=json.dumps(payload), headers=headers)
    #print(r.json())

    #POST method
    url='http://localhost:8983/solr/'+db+'/update/json?commit=true%27'
    r = requests.get(url)
    #print(r.json())

In [4]:
def get_all_tweets(screen_name):
    # initialize a list to hold all the tweepy Tweets
    alltweets = []
    
    # make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name,count=200)
    
    # save most recent tweets
    alltweets.extend(new_tweets)
    
    # save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1
    
    # keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        print("getting tweets before %s" % (oldest))
        
        # all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)
        
        # save most recent tweets
        alltweets.extend(new_tweets)
        
        # update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1
        
        print("...%s tweets downloaded so far" % (len(alltweets)))
        
    #Write into Solr   
    for tweet in alltweets:
        myTweet = {
            'id_str' : tweet.id_str,
            'user_id_str' : tweet.user.id_str,
            'created_at' : tweet.created_at.strftime('YYY-MM-DDThh:mm:ssZ'),
            'user' : {
                'id_str' : tweet.user.id_str,
                'screen_name' : tweet.user.screen_name,
                'followers_count' : tweet.user.followers_count,
                'friends_count' : tweet.user.friends_count
            },
            'text' : tweet.text,
            'favorite_count' : tweet.favorite_count,
            'in_reply_to_screen_name' : tweet.in_reply_to_screen_name,
            'retweet_count' : tweet.retweet_count,
            'retweeted' : tweet.retweeted
        }
        insert(myTweet, 'Tweets')
    print('Finished inserting tweets from '+screen_name)

In [5]:
# Get the users who have 40 or more followers
def simple_query():   
    connection = urlopen('http://localhost:8983/solr/Tweets/select?q=user.followers_count%3A%20%5B40%20TO%20*%5D')
    response = json.load(connection)
    for document in response['response']['docs']:
        print ("  Name =", document['user.screen_name'])

In [6]:
# Get the tweets with the word iPhone
def search_query():
    connection = urlopen('http://localhost:8983/solr/Tweets/select?q=text%3A%25iPhone%25')
    response = json.load(connection)
    for document in response['response']['docs']:
        print ("  Name =", document['text'])

In [7]:
if __name__ == '__main__':
    #get_all_tweets("srcaballero99") #pass in the username of the account you want to insert
    #get_all_tweets("Star_Lord987")
    #get_all_tweets("Rush2Loot")
    
    simple_query()
    search_query()

  Name = ['RT @BenGeskin: @LaughingQuoll Looks great on notchless iPhone 🔥\n\n(OnePlus 7 Pro) https://t.co/7H6ZU0MGwK']
  Name = ['RT @BenGeskin: @LaughingQuoll Looks great on notchless iPhone 🔥\n\n(OnePlus 7 Pro) https://t.co/7H6ZU0MGwK']
